In [3]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random

In [20]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer

In [4]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

In [8]:
import numpy as np

In [1]:
import nlpaug.augmenter.word as naw

In [8]:
aug = naw.SynonymAug(aug_src="wordnet")

In [9]:
def random_deletion(tokens, p=0.5):
    if len(tokens) == 0:
        return tokens

    mask = np.random.rand(len(tokens)) > p
    remaining_tokens = list(np.array(tokens)[mask])
    # remaining_tokens = [token for token in tokens if random.uniform(0, 1) > p]
    if len(remaining_tokens) == 0:
        return [random.choice(tokens)]  # 如果全部删除，则随机保留一个
    return remaining_tokens

# aug = nas.AbstSummAug(model_path='t5-base', device='cuda')
    
def t5_sum(tokens, aug, n=1):
    text = ' '.join(tokens)
    augmented_text = aug.augment(text, n)
    
    return augmented_text

In [5]:
TEXT = data.Field(tokenize='spacy',
                  include_lengths=True) # necessary for packed_padded_sequence
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [14]:
from nltk.tokenize import word_tokenize

In [22]:
stop_words = set(stopwords.words('english'))

In [25]:
'i' in stop_words

True

In [18]:
for example in train_data:
    text = example.text
    text = ' '.join(text)
    # print(text)
    aug_text = aug.augment(text)
    print(aug_text)
    text = word_tokenize(aug_text[0])
    print(text)
    break

["Well when watching this film late one night I was simple baffle by information technology ' s immensity. Fantastic script, great acting, costume and special effects, and the plot twists, wow! ! In fact if you lavatory see the terminate coming you should become a writer yourself. <br /> <br /> Great, I would recommend this film to anyone, especially if I don; deoxythymidine monophosphate like them much. <br /> <br /> Terrific"]
['Well', 'when', 'watching', 'this', 'film', 'late', 'one', 'night', 'I', 'was', 'simple', 'baffle', 'by', 'information', 'technology', "'", 's', 'immensity', '.', 'Fantastic', 'script', ',', 'great', 'acting', ',', 'costume', 'and', 'special', 'effects', ',', 'and', 'the', 'plot', 'twists', ',', 'wow', '!', '!', 'In', 'fact', 'if', 'you', 'lavatory', 'see', 'the', 'terminate', 'coming', 'you', 'should', 'become', 'a', 'writer', 'yourself', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'Great', ',', 'I', 'would', 'recommend', 'this', 'film', 'to', 'anyone', ',

In [31]:
TEXT = data.Field(tokenize='spacy', include_lengths=True)
LABEL = data.LabelField(dtype=torch.float)

In [33]:
TEXT.build_vocab(trainset.dataset, max_size=20000, vectors=f"glove.6B.100d")

In [27]:
trainset = TextDatasetWrapper(trainset.dataset)

In [21]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer

In [25]:
for i in trainset:
    print(i.label)
    break

pos


In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

# 移除停用词和进行词干提取
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(words):
    filtered_words = [word for word in words if (word.lower() not in stop_words) and (word not in string.punctuation)]
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    return stemmed_words

In [19]:
from tqdm.notebook import tqdm

In [20]:
new_train_data = []
for example in tqdm(train_data.examples, desc="Preprocess Trainset"):
    text = preprocess_text(example.text)
    label = example.label
    new_example = data.Example.fromlist([text, label], [('text', TEXT), ('label', LABEL)])
    new_train_data.append(new_example)
train_data.examples = new_train_data

new_test_data = []
for example in tqdm(test_data.examples, desc="Preprocess Testset"):
    text = preprocess_text(example.text)
    label = example.label
    new_example = data.Example.fromlist([text, label], [('text', TEXT), ('label', LABEL)])
    new_test_data.append(new_example)
test_data.examples = new_test_data

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/25000 [00:00<?, ?it/s]

In [ ]:
new_examples = []
# aug = nas.AbstSummAug(model_path='t5-base', device='cuda')
# aug = naw.SynonymAug(aug_src='ppdb', model_path='./nlpaug_model/ppdb-2.0-tldr')
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
for example in tqdm(train_data.examples):
    # 对原始文本进行随机删除数据增强
    # example_augmented_text = random_deletion(example.text)
    example_augmented_text = t5_sum(example.text, aug, 1)
    
    # 创建一个新的例子并将其添加到训练数据中
    new_example = data.Example.fromlist([example_augmented_text, example.label], [('text', TEXT), ('label', LABEL)])
    new_examples.append(new_example)
train_data.examples.extend(new_examples)

In [14]:
len(train_data.examples)

40000

In [21]:
import torchtext.vocab as vocab

TEXT.build_vocab(train_data,
                 max_size=VOCABULARY_SIZE,
                 # vectors='glove.6B.100d',
                 vectors=vocab.GloVe(name='6B', dim=100),
                 unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [22]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    device=DEVICE)

In [23]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    print(batch.text[0])
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([76, 128])
Target vector size: torch.Size([128])
tensor([[ 498,    8,   50,  ...,   63, 3910,  121],
        [  29,  314,  298,  ...,   24, 1093,   11],
        [  49,    3,  422,  ...,   18,  869,  115],
        ...,
        [   7,  303,  853,  ...,   15,    8,  670],
        [9142,  334,    3,  ...,   15,    3,    3],
        [  32,    3,    4,  ...,    1,    1,    1]], device='cuda:0')

Valid:
Text matrix size: torch.Size([61, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([25, 128])
Target vector size: torch.Size([128])


In [27]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.embedding.weight.data.copy_(TEXT.vocab.vectors)

        self.rnn = nn.LSTM(embedding_dim, hidden_dim)

        if output_dim is not None:
            self.fc = nn.Linear(hidden_dim, output_dim)
        else:
            self.fc = None
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length.cpu()).cuda()
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.rnn(packed)

        if self.fc is not None:
            return self.fc(hidden.squeeze(0)).view(-1)
        else:
            return hidden.squeeze(0)

In [28]:
INPUT_DIM = len(TEXT.vocab)

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [29]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100


In [30]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.text
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/157 | Cost: 0.6926
Epoch: 001/015 | Batch 050/157 | Cost: 0.6879
Epoch: 001/015 | Batch 100/157 | Cost: 0.6870
Epoch: 001/015 | Batch 150/157 | Cost: 0.5695
training accuracy: 69.82%
valid accuracy: 63.40%
Time elapsed: 0.06 min
Epoch: 002/015 | Batch 000/157 | Cost: 0.6028
Epoch: 002/015 | Batch 050/157 | Cost: 0.5619
Epoch: 002/015 | Batch 100/157 | Cost: 0.6247
Epoch: 002/015 | Batch 150/157 | Cost: 0.6169
training accuracy: 75.76%
valid accuracy: 58.18%
Time elapsed: 0.12 min
Epoch: 003/015 | Batch 000/157 | Cost: 0.5952
Epoch: 003/015 | Batch 050/157 | Cost: 0.5368
Epoch: 003/015 | Batch 100/157 | Cost: 0.4706
Epoch: 003/015 | Batch 150/157 | Cost: 0.4776
training accuracy: 78.40%
valid accuracy: 61.46%
Time elapsed: 0.18 min
Epoch: 004/015 | Batch 000/157 | Cost: 0.4826
Epoch: 004/015 | Batch 050/157 | Cost: 0.4834
Epoch: 004/015 | Batch 100/157 | Cost: 0.4913
Epoch: 004/015 | Batch 150/157 | Cost: 0.4758
training accuracy: 81.89%
valid accuracy: 58.74%

## No Aug

training accuracy: 90.28%
valid accuracy: 84.80%
Time elapsed: 1.28 min
Total Training Time: 1.28 min
Test accuracy: 84.42%

## Rand Del
training accuracy: 90.51%
valid accuracy: 85.56%
Time elapsed: 1.99 min
Total Training Time: 1.99 min
Test accuracy: 85.02%

## Synonym Aug
training accuracy: 66.77%
valid accuracy: 80.44%
Time elapsed: 1.50 min
Total Training Time: 1.50 min
Test accuracy: 79.55%

In [11]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [16]:


print('Probability positive:')
predict_sentiment(model, "Do i really love this movie? Yes I do")



Probability positive:


0.5735723376274109